In [1]:
import os
import shutil
import urllib3
import zipfile
import pandas as pd

In [62]:
http = urllib3.PoolManager()
url = 'http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)

with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:
    shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)
    

KeyboardInterrupt: 

In [2]:
!ls

'ls'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [3]:
lines = pd.read_csv('fra.txt', names=['src','tar','lic'], sep='\t')
del lines['lic']
len(lines)

192341

In [4]:
lines = lines.loc[:, 'src':'tar']
liens = lines[0:70000]
lines.tar = lines.tar.apply(lambda x:'\t'+ x + '\n')

lines[:10]

,src,tar
0,Go.,\tVa !\n
1,Go.,\tMarche.\n
2,Go.,\tBouge !\n
3,Hi.,\tSalut !\n
4,Hi.,\tSalut.\n
5,Run!,\tCours !\n
6,Run!,\tCourez !\n
7,Run!,\tPrenez vos jambes à vos cous !\n
8,Run!,\tFile !\n
9,Run!,\tFilez !\n


In [5]:
src_vocab = set()
for line in lines.src:
    for char in line:
        src_vocab.add(char)
        
tar_vocab=set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [6]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))

src_vocab_size = len(src_vocab) + 1
tar_vocab_size = len(tar_vocab) + 1

src_to_idx = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_idx = dict([(word, i+1) for i, word in enumerate(tar_vocab)])

print(src_to_idx)
print(tar_to_idx)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, '+': 8, ',': 9, '-': 10, '.': 11, '/': 12, '0': 13, '1': 14, '2': 15, '3': 16, '4': 17, '5': 18, '6': 19, '7': 20, '8': 21, '9': 22, ':': 23, ';': 24, '?': 25, 'A': 26, 'B': 27, 'C': 28, 'D': 29, 'E': 30, 'F': 31, 'G': 32, 'H': 33, 'I': 34, 'J': 35, 'K': 36, 'L': 37, 'M': 38, 'N': 39, 'O': 40, 'P': 41, 'Q': 42, 'R': 43, 'S': 44, 'T': 45, 'U': 46, 'V': 47, 'W': 48, 'X': 49, 'Y': 50, 'Z': 51, 'a': 52, 'b': 53, 'c': 54, 'd': 55, 'e': 56, 'f': 57, 'g': 58, 'h': 59, 'i': 60, 'j': 61, 'k': 62, 'l': 63, 'm': 64, 'n': 65, 'o': 66, 'p': 67, 'q': 68, 'r': 69, 's': 70, 't': 71, 'u': 72, 'v': 73, 'w': 74, 'x': 75, 'y': 76, 'z': 77, '\xa0': 78, '\xad': 79, '°': 80, 'º': 81, 'é': 82, 'ö': 83, 'ú': 84, '–': 85, '—': 86, '‘': 87, '’': 88, '₂': 89, '€': 90}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, '+': 12, ',': 13, '-': 14, '.': 15, '/': 16, '0': 17, '1': 18, '2': 19, '3': 20, '4': 21, '5': 22, 

In [7]:
encoder_input = []
for line in lines.src:
    encoder_input.append([src_to_idx[w] for w in line])
    
print(encoder_input[:5])

[[32, 66, 11], [32, 66, 11], [32, 66, 11], [33, 60, 11], [33, 60, 11]]


In [8]:
decoder_input = []
for line in lines.tar:
    decoder_input.append([tar_to_idx[w] for w in line])
    
print(decoder_input[:5])

[[1, 51, 56, 3, 4, 2], [1, 42, 56, 73, 58, 63, 60, 15, 2], [1, 31, 70, 76, 62, 60, 3, 4, 2], [1, 48, 56, 67, 76, 75, 3, 4, 2], [1, 48, 56, 67, 76, 75, 15, 2]]


In [9]:
decoder_target = []
for line in lines.tar:
    decoder_target.append([tar_to_idx[w] for w in line if w != '\t'])
    
print(decoder_target[:5])

[[51, 56, 3, 4, 2], [42, 56, 73, 58, 63, 60, 15, 2], [31, 70, 76, 62, 60, 3, 4, 2], [48, 56, 67, 76, 75, 3, 4, 2], [48, 56, 67, 76, 75, 15, 2]]


In [10]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])

encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [11]:
from tensorflow.keras.utils import to_categorical

encoder_input= to_categorical(encoder_input)
decoder_input=to_categorical(decoder_input)
decoder_target=to_categorical(decoder_target)

MemoryError: Unable to allocate 28.9 GiB for an array with shape (67511691, 115) and data type float32

In [12]:
from keras.layers import Input, LSTM

encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(256, return_state=True)

encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]


In [14]:
from keras.layers import Dense

decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)


In [16]:
from keras.models import Model

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [17]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=128, epochs=25, validation_split=0.2)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [ ]:
decoder_state_input_h = Input(shape=(256))
decoder_state_input_c = Input(shape=(256))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

decoder_states=[state_h, state_c]

decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs]+decoder_states)

In [ ]:
idx_to_src = dict((i,char)for char, i in src_to_idx.items())
idx_to_tar = dict((i,char)for char, i in src_to_idx.items())

In [ ]:
def predict_decode(input_seq):
    states_value = encoder_model.predict(input_seq)
    
    target_seq = np.zeros((1,1,tar_vocab_size))
    target_seq[0,0,tar_to_idx['\t']] = 1
    
    stop = False
    decoded_sentence = ""
    
    while not stop:
        output_tokens, h,c = decoder_model.predict([target_seq]+states_value)
        
        sampled_token_index = np.argmax(output_tokens[0,-1,:])
        sampled_char = idx_to_tar[sampled_token_index]
        
        decoded_sentence += sampled_char
        
        if sampled_char == '\n' or len(decoded_sentence) > max_tar_len):
            stop = True
            
        target_seq = np.zeros((1,1, tar_vocab_size))
        target_seq[0,0,sampled_token_index] = 1
        states_value = [h,c]
        
return decoded_sentence

In [ ]:
import numpy as np

for seq_index in [0,10,100,1000]:
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = predict_decode(input_seq)
    
    print('입력:', lines.src[seq_index])
    print('정답:', lines.src[seq_index][1:len(lines.tar[seq_index])-1])
    print('번역:', decoded_sentence[:len(decoded_sentence)-1], '\n')

In [ ]:
encoder_inputs = Input(shape(None, src_vocab_size))
encoder_lstm = LSTM(256, return_state=True)

encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

In [ ]:
import tensorflow as tf
from keras.layers import Attention
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)

S_ = tf.concat([state_h[:, tf.newaxis, :], decoder_outputs[:,:-1,:]], axis=1)

attention= Attention()
context_vector = attention([S_, encoder_outputs])
concat= tf.concat([decoder_outputs, context_vector], axis=-1)
decoder_sofrmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(concat)


model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy)

In [ ]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=128, epochs=25, validation_split=0.2)

In [ ]:
encoder_model = Model(inputs=encoder_inputs, outputs=[encoder_states])

In [ ]:
decoder_state_input_h = Input(shape=(256))
decoder_state_input_c = Input(shape=(256))

estate_h = Input(shape=(256))
encoder_outputs = Input(shape=(256))


decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

decoder_states=[state_h, state_c]

S_ tf.concat([estate_h[:, tf.newaxis,:], decoder_outputs[:,:-1,:]], axis=1)
context_vector = attention([S_, encoder_outputs])
decoder_concat = tf.concat([decoder_outputs, context_vector], axis=-1)
decoder_outputs = decoder_softmax_layer(decoder_concat)
decoder_model = Model(inputs=[decoder_inputs, estate_h, encoder_outputs] + decoder_states_inputs, outputs=[decoder_outputs]+decoder_states)